In [1]:
!python3 -m pip install --upgrade pip

In [2]:
# !pip install openai==0.27.8
!pip install openai==1.2.4

In [3]:
!pip install python-dotenv tiktoken

In [4]:
!pip install pdfplumber

In [5]:
import openai
import pdfplumber
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv
import os
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
client = OpenAI()

In [7]:
# アシスタントの作成
assistant = client.beta.assistants.create(
    instructions="あなたはプログラムコードを書くシステムエンジニアです。質問を実現するpythonコードを書いて実行します。",
    model="gpt-4-1106-preview",
    tools=[{"type": "code_interpreter"}],
)

In [8]:
# 会話するためスレッドの準備
thread = client.beta.threads.create()

In [9]:
# ユーザーメッセージの追加
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="日本プロ野球の選手データを取得し、選手の平均年齢を出力するコードを作成したいです。手伝ってください。"
)

In [10]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)

for message in messages:
    print(message.role, ":", message.content[0].text.value)

user : 日本プロ野球の選手データを取得し、選手の平均年齢を出力するコードを作成したいです。手伝ってください。


In [11]:
# アシスタントにリクエスト
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [12]:
# 実行状況の確認
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)

completed


In [13]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)

user : 日本プロ野球の選手データを取得し、選手の平均年齢を出力するコードを作成したいです。手伝ってください。
assistant : インターネットアクセスが無効になっているため、直接日本プロ野球の選手データを取得することはできません。しかし、仮のデータセットを作成して、その上で選手の平均年齢を計算するコードを提供することは可能です。もし実際のデータを持っているなら、それをアップロードしていただければ、そのデータを使って計算することもできます。

まずは、仮の選手データの例を作成してその上で平均年齢を計算するサンプルコードを書いていきます。これはあくまでも例ですので、実際のデータに対しては適切なデータ処理が必要になります。ここでは、各選手について名前と生年月日が記録されたシンプルなデータを想定します。


In [14]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_hmgVN8DputUCeD3ExUAICy8t', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='日本プロ野球の選手データを取得し、選手の平均年齢を出力するコードを作成したいです。手伝ってください。'), type='text')], created_at=1700315911, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_F5cAI7h3htu1p7wC4Dhd0flb'), ThreadMessage(id='msg_glR2l7nee7UvKARFNgJlPTeY', assistant_id='asst_CWbof6eUqGW97YdaZm8Z2nj9', content=[MessageContentText(text=Text(annotations=[], value='インターネットアクセスが無効になっているため、直接日本プロ野球の選手データを取得することはできません。しかし、仮のデータセットを作成して、その上で選手の平均年齢を計算するコードを提供することは可能です。もし実際のデータを持っているなら、それをアップロードしていただければ、そのデータを使って計算することもできます。\n\nまずは、仮の選手データの例を作成してその上で平均年齢を計算するサンプルコードを書いていきます。これはあくまでも例ですので、実際のデータに対しては適切なデータ処理が必要になります。ここでは、各選手について名前と生年月日が記録されたシンプルなデータを想定します。'), type='text')], created_at=1700315931, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_CkAuHkFjOyWiaevimC52rYIg', thread_id='thread_F5cAI7

In [15]:
# Run Stepの確認
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)
print(run_steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_NpFyNBlJ6xpJyX0wHPD7t9vH', assistant_id='asst_CWbof6eUqGW97YdaZm8Z2nj9', cancelled_at=None, completed_at=1700315937, created_at=1700315931, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_CkAuHkFjOyWiaevimC52rYIg', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_glR2l7nee7UvKARFNgJlPTeY'), type='message_creation'), thread_id='thread_F5cAI7h3htu1p7wC4Dhd0flb', type='message_creation', expires_at=None)], object='list', first_id='step_NpFyNBlJ6xpJyX0wHPD7t9vH', last_id='step_NpFyNBlJ6xpJyX0wHPD7t9vH', has_more=False)


In [17]:
# ユーザーメッセージの追加
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="では、仮のデータセットを作成してそのうえで選手の平均年齢を計算するサンプルコードを作成してください"
)

# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)

for message in messages:
    print(message.role, ":", message.content[0].text.value)

# アシスタントにリクエスト
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# 実行状況の確認
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)

# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)

user : 日本プロ野球の選手データを取得し、選手の平均年齢を出力するコードを作成したいです。手伝ってください。
assistant : インターネットアクセスが無効になっているため、直接日本プロ野球の選手データを取得することはできません。しかし、仮のデータセットを作成して、その上で選手の平均年齢を計算するコードを提供することは可能です。もし実際のデータを持っているなら、それをアップロードしていただければ、そのデータを使って計算することもできます。

まずは、仮の選手データの例を作成してその上で平均年齢を計算するサンプルコードを書いていきます。これはあくまでも例ですので、実際のデータに対しては適切なデータ処理が必要になります。ここでは、各選手について名前と生年月日が記録されたシンプルなデータを想定します。
user : では、仮のデータセットを作成してそのうえで選手の平均年齢を計算するサンプルコードを作成してください
in_progress
user : 日本プロ野球の選手データを取得し、選手の平均年齢を出力するコードを作成したいです。手伝ってください。
assistant : インターネットアクセスが無効になっているため、直接日本プロ野球の選手データを取得することはできません。しかし、仮のデータセットを作成して、その上で選手の平均年齢を計算するコードを提供することは可能です。もし実際のデータを持っているなら、それをアップロードしていただければ、そのデータを使って計算することもできます。

まずは、仮の選手データの例を作成してその上で平均年齢を計算するサンプルコードを書いていきます。これはあくまでも例ですので、実際のデータに対しては適切なデータ処理が必要になります。ここでは、各選手について名前と生年月日が記録されたシンプルなデータを想定します。
user : では、仮のデータセットを作成してそのうえで選手の平均年齢を計算するサンプルコードを作成してください
assistant : 


In [18]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_hmgVN8DputUCeD3ExUAICy8t', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='日本プロ野球の選手データを取得し、選手の平均年齢を出力するコードを作成したいです。手伝ってください。'), type='text')], created_at=1700315911, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_F5cAI7h3htu1p7wC4Dhd0flb'), ThreadMessage(id='msg_glR2l7nee7UvKARFNgJlPTeY', assistant_id='asst_CWbof6eUqGW97YdaZm8Z2nj9', content=[MessageContentText(text=Text(annotations=[], value='インターネットアクセスが無効になっているため、直接日本プロ野球の選手データを取得することはできません。しかし、仮のデータセットを作成して、その上で選手の平均年齢を計算するコードを提供することは可能です。もし実際のデータを持っているなら、それをアップロードしていただければ、そのデータを使って計算することもできます。\n\nまずは、仮の選手データの例を作成してその上で平均年齢を計算するサンプルコードを書いていきます。これはあくまでも例ですので、実際のデータに対しては適切なデータ処理が必要になります。ここでは、各選手について名前と生年月日が記録されたシンプルなデータを想定します。'), type='text')], created_at=1700315931, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_CkAuHkFjOyWiaevimC52rYIg', thread_id='thread_F5cAI7